# Twitter Sentiment Analysis - Fake Covid-19 dataset

We've used the following packages:

In [1]:
import plotly.express as px
#from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import json
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import emoji
from nltk.corpus import stopwords
import itertools
import altair as alt
from dateutil.parser import parse
from IPython.core.display import Image, display

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Then we have defined the following functions to clean the tweets' text:

In [2]:
csv_dataframe = pd.read_csv('fakecovid/dataset/FINAL_fakecovid_final_filtered_dataset_clean.csv',sep=";")
csv_dataframe['tweet_id'] = csv_dataframe['tweet_id'].astype(str)
csv_list = csv_dataframe.values.tolist()
lista_unica_csv=list(itertools.chain.from_iterable(csv_list))

data = []
with open('fakecovid/dataset/fakecovid_result_final_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

We'll count how many #covid19 hashatags are positive, neutral or negative.

In [13]:
hashtag_list = []
neutral_list_false = []
negative_list_false = []
positive_list_false = []

neutral_list_pfalse = []
negative_list_pfalse = []
positive_list_pfalse = []

index=0

for element in data:
    token_id = data[index]['id_str']
    indice_csv = lista_unica_csv.index(token_id)
    
    for entity in data[index]['entities']['hashtags']:
        if entity['text'] == 'covid19':
            hashtag_list.append(data[index]['full_text'])
            score = SentimentIntensityAnalyzer().polarity_scores(data[index]['full_text'])
        
            if lista_unica_csv[indice_csv+1].lower() == "false":
                neg = score['neg']
                neu = score['neu']
                pos = score['pos']
                
                if neg > pos:
                    negative_list_false.append(data[index]['full_text'])
        
                elif pos > neg:
                    positive_list_false.append(data[index]['full_text'])
        
                elif pos == neg:
                    neutral_list_false.append(data[index]['full_text'])
            
            elif lista_unica_csv[indice_csv+1].lower() == "partially false":
                neg = score['neg']
                neu = score['neu']
                pos = score['pos']
                
                if neg > pos:
                    negative_list_pfalse.append(data[index]['full_text'])

                elif pos > neg:
                    positive_list_pfalse.append(data[index]['full_text'])
        
                elif pos == neg:
                    neutral_list_pfalse.append(data[index]['full_text'])
          
    index=index+1

In [14]:
print("Total number of tweets containing #covid19: ",len(hashtag_list))
print("\nPositive number (False): ",len(positive_list_false))
print("Negative number (False): ", len(negative_list_false))
print("Neutral number (False): ",len(neutral_list_false))

print("\nPositive number (Partially False): ",len(positive_list_pfalse))
print("Negative number (Partially False): ", len(negative_list_pfalse))
print("Neutral number (Partially False): ",len(neutral_list_pfalse))

category = []

categorypositivefalse = ["Positive false"] * len(positive_list_false)
categoryneutralfalse = ["Neutral false"] * len(neutral_list_false)
categorynegativefalse = ["Negative false"] * len(negative_list_false)

categorypositivepfalse = ["Positive partiallyfalse"] * len(positive_list_pfalse)
categoryneutralpfalse = ["Neutral partiallyfalse"] * len(neutral_list_pfalse)
categorynegativepfalse = ["Negative partiallyfalse"] * len(negative_list_pfalse)

category = categorypositivefalse + categoryneutralfalse + categorynegativefalse + categorypositivepfalse + categoryneutralpfalse + categorynegativepfalse

fdist = dict(nltk.FreqDist(category))

df_fake5 = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
df_fake5 = df_fake5.rename(columns={'index':'Category', 0:'Count'})

Total number of tweets containing #covid19:  3

Positive number (False):  0
Negative number (False):  1
Neutral number (False):  1

Positive number (Partially False):  1
Negative number (Partially False):  0
Neutral number (Partially False):  0


In [15]:
col_one_list = df_fake5['Category'].tolist()
col_two_list = df_fake5['Count'].tolist()
typelist=[]
namelist=[]

index = 0

count_false = [0] * len(col_one_list)
count_part = [0] * len(col_one_list)

for el in col_one_list:
    tok = el.split()
    namelist.append(tok[0])
    if tok[0] in namelist:
        indx = namelist.index(tok[0])
        if tok[1] == "false":
            count_false[indx] = col_two_list[index]
        elif tok[1] == "partiallyfalse":
            count_part[indx] = col_two_list[index]
        else:
            print("errore count")
    index = index + 1

i=0
for el in col_two_list:
    col_two_list[i] = count_false[i] + count_part[i]
    i = i + 1

df_fake5['Category']=namelist
df_fake5['False']=count_false
df_fake5['Partially False']=count_part
del df_fake5['Count']
df_fake5['Count'] = df_fake5['False'] + df_fake5['Partially False']

range_ = ["#0C7BDC","#FFC20A"]


bars = alt.Chart(df_fake5).transform_fold(
    ['False', 'Partially False']
).mark_bar().encode(
    x=alt.X('key:N',title=None,axis=alt.Axis(labels=False)),
    y=alt.Y('value:Q',title="Count"),
    color=alt.Color('key:N', scale=alt.Scale(range=range_),title="Category"),
    column= alt.Column('Category:N', title="Sentiment")
).properties(
    title="Sentiment analysis on #covid19 - Fake Covid-19 dataset", 
    width=75
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    titlePadding=10
).configure_header(
    titleFontSize=15,
    labelFontSize=12
)

bars

alt.Chart(...)